In [1]:
import sympy as sym
import numpy as np

In [2]:
t = sym.Symbol('t')

I = sym.Symbol('I')
m = sym.Symbol('m')
m_inf = sym.Symbol('m_inf')

p = sym.Symbol('p')

prestep_V = sym.Symbol('prestep_V')
step_V = sym.Symbol('step_V')

a = sym.Symbol('a')
b = sym.Symbol('b')

g_max = sym.Symbol('g_max')
E_rev = sym.Symbol('E_rev')
M_ma = sym.Symbol('M_ma')
M_mb = sym.Symbol('M_mb')
V_2ma = sym.Symbol('V_2ma')
V_2mb = sym.Symbol('V_2mb')
s_ma = sym.Symbol('s_ma')
s_mb = sym.Symbol('s_mb')

In [3]:
# H-H model
m_inf = 1 / (1 + M_mb/M_ma * ((1+sym.exp((prestep_V-V_2ma)/s_ma)) / (1+sym.exp((prestep_V-V_2mb)/s_mb))))

a = M_ma / (1 + sym.exp((step_V - V_2ma) / s_ma))
b = M_mb / (1 + sym.exp((step_V - V_2mb) / s_mb))

m = (a - (a-(a+b)*m_inf)*sym.exp(-t*(a+b))) / (a + b)
I = g_max * m ** 4 * (step_V - E_rev)

In [4]:
# subs 
t_points = np.arange(0.0, 120, 0.01) 
sim_p = 4

sim_prestep_V = -100
prestep_Vs = [-80, -50, -20]

step_Vs = [0.00, 10.00, 20.00, 30.00, 40.00, 50.00, 60.00, 70.00, 80.00]
sim_step_V = 80

traces_Vs = [(sim_prestep_V, v) for v in step_Vs]
traces_Vs.extend([(v, sim_step_V) for v in prestep_Vs])
traces_Vs

[(-100, 0.0),
 (-100, 10.0),
 (-100, 20.0),
 (-100, 30.0),
 (-100, 40.0),
 (-100, 50.0),
 (-100, 60.0),
 (-100, 70.0),
 (-100, 80.0),
 (-80, 80),
 (-50, 80),
 (-20, 80)]

In [ ]:
# read dataset_test
import pandas as pd

params_test = pd.read_csv('dataset_test.csv').iloc[:, -8:]
params_test.columns

In [ ]:
# Initialize a sensitivity_coef_matrix with zeros
num_matrices = params_test.shape[0]
num_params = 8


sensitivity_coef_matrices = np.zeros((num_matrices, len(traces_Vs) * len(t_points), num_params))
sensitivity_coef_matrices.shape


In [ ]:
params_sub = {g_max:0, E_rev:0, M_ma:0, M_mb:0, V_2ma:0, V_2mb:0, s_ma:0, s_mb:0}

In [ ]:
list(params_sub.keys())

In [ ]:
for i in range(1): # iterating thru all matrices/samples
    col = 0
    
    params_sub = {g_max:0, E_rev:0, M_ma:0, M_mb:0, V_2ma:0, V_2mb:0, s_ma:0, s_mb:0}
    for p in list(params_sub.keys()):   # assign the subs of parameters of ith sample
        params_sub[p] = params_test[str(p)][i]


    for param in list(params_sub.keys()):   # iterate thru param to be integrated
        row = 0

        # params_sub.pop(param, None) # remove the integrated param from params to be subbed
        # for p in list(params_sub.keys()):   # assign the subs of parameters of ith sample
        #     params_sub[p] = params_test[str(p)][i]


        # log entries on a col
        for point in t_points: 
            params_sub[t] = point
            for trace in traces_Vs: 
                params_sub[prestep_V] = trace[0]
                params_sub[step_V] = trace[1]
                
                sensitivity_coef_matrices[i, row, col] = sym.diff(I, param).subs(params_sub).evalf()

                row += 1
        
        col += 1
        # params_sub = {g_max:0, E_rev:0, M_ma:0, M_mb:0, V_2ma:0, V_2mb:0, s_ma:0, s_mb:0}

In [ ]:
param

In [ ]:
params_sub

In [ ]:
sensitivity_coef_matrices.shape

In [ ]:
sensitivity_coef_matrices[0,:,0]

In [ ]:
norms = np.linalg.norm(sensitivity_coef_matrices[0], axis=0)

print("2-norms of each column vector:")
print(norms)

In [25]:
substitution = {prestep_V:-100, step_V:80, E_rev:65, t:2, M_ma:1, M_mb:5, V_2ma:34, V_2mb:23, s_ma:0.3, s_mb:0.5}

In [15]:
dI_dg.subs({prestep_V:-100, step_V:80, E_rev:65, t:2, M_ma:1, M_mb:5, V_2ma:34, V_2mb:23, s_ma:0.3, s_mb:0.5}).evalf()

0.0115740740740741

In [5]:
(s_ma / I) * dI_ds_ma

s_ma*(M_ma/(exp((-V_2ma + step_V)/s_ma) + 1) + M_mb/(exp((-V_2mb + step_V)/s_mb) + 1))**4*(-4*M_ma*g*(-E_rev + step_V)*(-V_2ma + step_V)*(M_ma/(exp((-V_2ma + step_V)/s_ma) + 1) - (M_ma/(exp((-V_2ma + step_V)/s_ma) + 1) - (M_ma/(exp((-V_2ma + step_V)/s_ma) + 1) + M_mb/(exp((-V_2mb + step_V)/s_mb) + 1))/(1 + M_mb*(exp((-V_2ma + prestep_V)/s_ma) + 1)/(M_ma*(exp((-V_2mb + prestep_V)/s_mb) + 1))))*exp(-t*(M_ma/(exp((-V_2ma + step_V)/s_ma) + 1) + M_mb/(exp((-V_2mb + step_V)/s_mb) + 1))))**4*exp((-V_2ma + step_V)/s_ma)/(s_ma**2*(M_ma/(exp((-V_2ma + step_V)/s_ma) + 1) + M_mb/(exp((-V_2mb + step_V)/s_mb) + 1))**5*(exp((-V_2ma + step_V)/s_ma) + 1)**2) + g*(-E_rev + step_V)*(M_ma/(exp((-V_2ma + step_V)/s_ma) + 1) - (M_ma/(exp((-V_2ma + step_V)/s_ma) + 1) - (M_ma/(exp((-V_2ma + step_V)/s_ma) + 1) + M_mb/(exp((-V_2mb + step_V)/s_mb) + 1))/(1 + M_mb*(exp((-V_2ma + prestep_V)/s_ma) + 1)/(M_ma*(exp((-V_2mb + prestep_V)/s_mb) + 1))))*exp(-t*(M_ma/(exp((-V_2ma + step_V)/s_ma) + 1) + M_mb/(exp((-V_2mb + 

In [7]:
m_inf.subs({prestep_V:-100}).evalf()

1/(1.0 + M_mb*(exp((-V_2ma - 100)/s_ma) + 1.0)/(M_ma*(exp((-V_2mb - 100)/s_mb) + 1.0)))

In [6]:
dI_dg.subs({step_V: 20, m_inf: m_inf})

(20 - E_rev)*(M_ma/(exp((20 - V_2ma)/s_ma) + 1) - (M_ma/(exp((20 - V_2ma)/s_ma) + 1) - (M_ma/(exp((20 - V_2ma)/s_ma) + 1) + M_mb/(exp((20 - V_2mb)/s_mb) + 1))/(1 + M_mb*(exp((-V_2ma + prestep_V)/s_ma) + 1)/(M_ma*(exp((-V_2mb + prestep_V)/s_mb) + 1))))*exp(-t*(M_ma/(exp((20 - V_2ma)/s_ma) + 1) + M_mb/(exp((20 - V_2mb)/s_mb) + 1))))**4/(M_ma/(exp((20 - V_2ma)/s_ma) + 1) + M_mb/(exp((20 - V_2mb)/s_mb) + 1))**4